In [1]:
import fm2p

Loading DLC 2.3.7...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


c:\Users\dmartins\anaconda3\envs\fm1\lib\site-packages\oasis\functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
eye_dlc_h5 = r'K:\Mini2P\250218_DMM_DMM038_pillar\250218_DMM_DMM038_rec\250218_DMM_DMM038_rec_01_eyecam_deinterDLC_resnet50_freely_moving_eyecams_02Feb12shuffle1_100000.h5'
eye_TS_csv = r'K:\Mini2P\250218_DMM_DMM038_pillar\250218_DMM_DMM038_rec\250218_DMM_DMM038_rec_01_eyecam.csv'
eye_TTLV_csv = r'K:\Mini2P\250218_DMM_DMM038_pillar\250218_DMM_DMM038_rec\250218_DMM_DMM038_rec_01_logTTL.csv'
eye_TTLTS_csv = r'K:\Mini2P\250218_DMM_DMM038_pillar\250218_DMM_DMM038_rec\250218_DMM_DMM038_rec_01_ttlTS.csv'

In [ ]:
# start time should be ~60 sec into the video

In [4]:
pts, _ = fm2p.open_dlc_h5(eye_dlc_h5)

In [12]:
num_frames = pts['t_x'].size
eyeT = fm2p.read_timestamp_file(eye_TS_csv, position_data_length=num_frames)

In [15]:
eyeT

array([41673.975731 , 41673.9905855, 41674.00544  , ..., 41934.254233 ,
       41934.278617 , 41934.29463  ])

In [19]:
ttlT_series = pd.read_csv(eye_TTLTS_csv, encoding='utf-8', engine='c', header=None).squeeze()
ttlT = fm2p.read_timestamp_series(ttlT_series)

In [ ]:
ttlV = pd.read_csv(eye_TTLV_csv, encoding='utf-8', engine='c', header=None).squeeze().to_numpy()

In [ ]:
print(len(ttlV), len(ttlT))

13646 13646


In [ ]:
startInd = int(np.argwhere(ttlV>0)[0])
startInd

3242

In [ ]:
endInd = int(np.argwhere(ttlV>0)[-1])
endInd

13342

In [36]:
reye_cam = fm2p.Eyecam('', '')
reye_cam.add_files(
    eye_dlc_h5=eye_dlc_h5,
    eye_avi='',
    eyeT=eye_TS_csv
)
eye_xyl, ellipse_dict = reye_cam.track_pupil()

100%|██████████| 15602/15602 [00:01<00:00, 9455.78it/s]


LinAlg error count = 136
boolean index did not match indexed array along dimension 1; dimension is 15602 but corresponding boolean dimension is 157
Error in scale, center, and calibration figures. Skipping these for now


In [40]:
theta = ellipse_dict['theta']

In [45]:
firstTheta = int(np.argwhere(~np.isnan(theta))[0])
lastTheta = int(np.argwhere(~np.isnan(theta))[-1])
print(firstTheta, lastTheta)

3747 15258


In [50]:
print('Theta: ', eyeT[firstTheta], ' to ', eyeT[lastTheta])
print('TTL: ', ttlT[startInd], ' to ', ttlT[endInd])

Theta:  41736.493222  to  41928.565158
TTL:  41736.462144  to  41928.514598


In [ ]:
apply_t0 = ttlT[startInd]
apply_tEnd = ttlT[endInd]

In [59]:
def find_closest_timestamp(arr, t):

    ind = np.argmin(np.abs(arr - t))
    approx_t = arr[ind]

    return ind, approx_t

In [62]:
eyeStart, _ = find_closest_timestamp(eyeT, apply_t0)
eyeEnd, _ = find_closest_timestamp(eyeT, apply_tEnd)

In [63]:
eyeStart, eyeEnd

(3745, 15255)